## Imports

In [1869]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

#### Importing data

In [1870]:
campaigns = pd.read_csv(r'C:\Users\omgit\Downloads\GROUP ASSIGNMENT DSC use-case-20241205\Data\campaigns.csv',sep=';')
donors = pd.read_csv(r'C:\Users\omgit\Downloads\GROUP ASSIGNMENT DSC use-case-20241205\Data\donors.csv')
gifts = pd.read_csv(r'C:\Users\omgit\Downloads\GROUP ASSIGNMENT DSC use-case-20241205\Data\gifts.csv')
campaign6169 = pd.read_csv(r'C:\Users\omgit\Downloads\GROUP ASSIGNMENT DSC use-case-20241205\Data\selection campaign 6169.csv')
campaign7244 = pd.read_csv(r'C:\Users\omgit\Downloads\GROUP ASSIGNMENT DSC use-case-20241205\Data\selection campaign 7244.csv')


#### Data Examination

In [ ]:
donors.head()

#### Checking for missing values

In [ ]:
missing_values = donors.isnull().sum()
print("Missing Values per Column:\n", missing_values)

#### Filling missing values in the zipcode column

In [1873]:
donors['zipcode'] = donors['zipcode'].fillna(-1)

#### Type conversion

In [1874]:
donors['zipcode'] = donors['zipcode'].astype(int)

converting dateofbirth to datetime , dropping rows with with null date of birth values

In [1875]:
# Converting 'dateOfBirth' to datetime type
donors['dateOfBirth'] = pd.to_datetime(donors['dateOfBirth'], format='%m/%d/%Y', errors='coerce')

# Dropping rows with invalid dates
donors = donors.dropna(subset=['dateOfBirth'])

# Creating a flag variable to indicate if gender was missing
donors['gender_missing'] = donors['gender'].isna().astype(int)

# Replacing missing gender values with the mode
gender_mode = donors['gender'].mode()[0]
donors['gender'] = donors['gender'].fillna(gender_mode)



In [ ]:
missing_values = donors.isnull().sum()
print("Missing Values per Column:\n", missing_values)

In [1877]:
# One-hot encode the 'language' column
donors = pd.get_dummies(donors, columns=['language'], prefix='lang',dtype=int,drop_first=True)



We chose a time gap of 1 month from the drop date of 4th September 2019. That is August 4 2018. From that point we take the last 5 years of data to maintain uniformity.

In [ ]:
#cutoff date is 4 aug 2018
cutoff_date = pd.Timestamp("2018-08-04")  # Corrected date format
cutin_date = cutoff_date - pd.DateOffset(years=5)

print("Cutoff Date:", cutoff_date)
print("Cut-in Date:", cutin_date)







Calculating age based on the cut-off date.

In [1879]:
donors['age'] = (cutoff_date - donors['dateOfBirth']).dt.days // 365

Checking for unrealistic ages

In [1880]:
# Checking for unrealistic ages (e.g., negative or very large values)
unrealistic_ages = donors[(donors['age'] < 0) | (donors['age'] > 120)]

In [ ]:
unrealistic_ages.head()

Filtering for unrealistic ages

In [1882]:
donors_cleaned = donors[donors['age'] <= 120]

Binning age

In [ ]:
min_age = 20
max_age = 120




bins = list(range(int(min_age // 10) * 10, int(max_age // 10 + 1) * 10 + 1, 10))  
labels = [f'{bins[i]}-{bins[i+1]-1}' for i in range(len(bins) - 1)]  


donors_cleaned['age_group'] = pd.cut(donors_cleaned['age'], bins=bins, labels=labels, right=False)



Checking the cleaned age group distribution

In [ ]:
donors_cleaned['age_group'].value_counts(dropna=False)

In [ ]:
missing_values = donors_cleaned.isnull().sum()
print("Missing Values per Column:\n", missing_values)

Dummies for region, age group and province

In [1886]:
donors_cleaned = pd.get_dummies(donors_cleaned, columns=['region'], prefix='reg',dtype=int,drop_first=True)
donors_cleaned = pd.get_dummies(donors_cleaned, columns=['age_group'], prefix='age_group',dtype=int,drop_first=True)
donors_cleaned = pd.get_dummies(donors_cleaned, columns=['province'], prefix='province',dtype=int,drop_first=True)


In [ ]:
donors_cleaned.head()

Flag variable for gender

In [1888]:
donors_cleaned['gender_flag'] = donors_cleaned['gender'].map({'M': 1, 'F': 0})

In [ ]:
true_count = donors_cleaned['reg_Missing'].sum()
true_count

Dropping irrelevant columns

In [1891]:
donors_cleaned = donors_cleaned.drop(columns = ['gender','dateOfBirth','age'])

Now we will clean the gifts dataset and create some features

In [ ]:
gifts.head()

Filling Missing values for campaignID , converting date into datetime and making amount into a useable column.

In [1894]:
gifts['campaignID']= gifts['campaignID'].fillna(-1)
gifts['date'] = pd.to_datetime(gifts['date'], errors='coerce',format = '%d/%m/%Y')
gifts['amount'] = gifts['amount'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)




In [ ]:
gifts.head()

In [ ]:
gifts.dtypes

In [ ]:
missing_values = gifts.isnull().sum()
print("Missing Values per Column:\n", missing_values)

Converting amount to numeric

In [1898]:
gifts['amount'] = pd.to_numeric(gifts['amount'],errors='coerce')

Filtering based on cutoff and cut-in days

In [1899]:
gifts_before_camp1cutoff = gifts[(gifts['date'] < cutoff_date) & (gifts['date'] > cutin_date)]


RFM features engineered

In [1901]:
frequency_df = gifts_before_camp1cutoff.groupby('donorID').size().reset_index(name='no_of_donations')
monetary_df = gifts_before_camp1cutoff.groupby('donorID')['amount'].sum().reset_index(name='sum_of_donations')
recency_df = gifts_before_camp1cutoff.groupby('donorID')['date'].max().reset_index(name = 'last_donation_date')


In [ ]:
frequency_df.head()

In [ ]:
monetary_df.head()

In [ ]:
recency_df.head()

Creating recency columns for each donor

In [1905]:
recency_df['days_since_last_donation'] = cutoff_date - recency_df['last_donation_date']

In [ ]:
recency_df.head()

Merging all of them together

In [1907]:
merged_df_pre_campaign = pd.merge(frequency_df,monetary_df,on= 'donorID')
merged_df_pre_campaign = pd.merge(merged_df_pre_campaign,recency_df,on= 'donorID')

Dropping unnecessary columns

In [ ]:
merged_df_pre_campaign = merged_df_pre_campaign.drop(columns='last_donation_date')
merged_df_pre_campaign.head()

Binning days_since_last_donation so we can onehot encode them.

In [ ]:
# Adjust bins and labels for a 5-year range
days_bins = [0, 90, 180, 270, 360, 450, 540, 630, 720, 810, 900, 990, 1080, 1170, 1260, 1350, 1440, 1530, 1620, 1710, 1825, float('inf')]
days_labels = [
    '0-3 months', '3-6 months', '6-9 months', '9-12 months', 
    '12-15 months', '15-18 months', '18-21 months', '21-24 months', 
    '24-27 months', '27-30 months', '30-33 months', '33-36 months', 
    '36-39 months', '39-42 months', '42-45 months', '45-48 months', 
    '48-51 months', '51-54 months', '54-57 months', '57-60 months', '60+ months'
]

# Apply the updated bins
merged_df_pre_campaign['days_bins'] = pd.cut(
    merged_df_pre_campaign['days_since_last_donation'].dt.days,
    bins=days_bins,
    labels=days_labels,
    right=False
)

# Display the updated DataFrame
print(merged_df_pre_campaign[['days_since_last_donation', 'days_bins']].head())


Binning sum of donations as well

In [ ]:
bin_edges = list(range(0, 181, 10)) + [float('inf')]  # Add 10-based bins and a final bin for 180+
bin_labels = [f'{bin_edges[i]}-{bin_edges[i + 1] - 1}' for i in range(len(bin_edges) - 2)] + ['180+']  # Create labels

# Apply the bins to create 'donation_value_bins'
merged_df_pre_campaign['donation_value_bins'] = pd.cut(
    merged_df_pre_campaign['sum_of_donations'],
    bins=bin_edges,  # Use manually defined bin edges
    labels=bin_labels,  # Use manually defined bin labels
    right=False  # Bin intervals are left-inclusive
)

# Display the resulting DataFrame
print(merged_df_pre_campaign[['sum_of_donations', 'donation_value_bins']].head())


In [ ]:
merged_df_pre_campaign[['donation_value_bins']].value_counts()

In [ ]:
merged_df_pre_campaign['donation_value_bins'].value_counts(dropna=False)

Creating dummies

In [1915]:
merged_df_pre_campaign = pd.get_dummies(merged_df_pre_campaign, columns=['days_bins'], prefix='days_bins',dtype=int)
merged_df_pre_campaign = pd.get_dummies(merged_df_pre_campaign, columns=['donation_value_bins'], prefix='donation_value_bins',dtype=int)

Dropping irrelevant columns

In [1916]:
merged_df_pre_campaign= merged_df_pre_campaign.drop(columns=['sum_of_donations','days_since_last_donation'])

In [ ]:
gifts_before_camp1cutoff.head()

Bringing it together

In [1919]:
train_selected_donors = campaign6169.merge(donors_cleaned, on='donorID', how='inner')

In [ ]:
train_selected_donors.head()

In [ ]:
missing_values = train_selected_donors.isnull().sum()
print("Missing Values per Column:\n", missing_values)

Creating a filtered table of all the donors who did donate for the campaign

In [1922]:
gifts_filtered = gifts[(gifts['campaignID'] == 6169) & (gifts['amount'] >= 30)]

Creating a flag variable for whether the campaign was a success or not

In [1923]:
train_selected_donors['campaign6169success'] = train_selected_donors['donorID'].isin(gifts_filtered['donorID']).astype(int)

In [ ]:
train_selected_donors

In [ ]:
missing_values = train_selected_donors.isnull().sum()
print("Missing Values per Column:\n", missing_values)

Merging additional info for those who were active before hand

In [1926]:
train_final = train_selected_donors.merge(merged_df_pre_campaign, on= 'donorID', how = 'left')

In [1927]:
train_final['no_of_donations']= train_final['no_of_donations'].fillna(0)
# train_final['days_since_last_donation']= train_final['days_since_last_donation'].fillna(-1)
# train_final['sum_of_donations']= train_final['sum_of_donations'].fillna(0)

In [ ]:
train_final.isna().sum()

In [1929]:
train_final = train_final.fillna(0)

In [ ]:
train_final.isna().sum()

In [ ]:
# Create a flag variable for null values in 'no_of_donations'
train_final['active_in_last_five_years'] = train_final['no_of_donations'].notnull().astype(int)

train_final

In [1932]:

train_final.to_csv('train_final.csv')